In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
import re
import emoji

# 데이터 로드
df = pd.read_csv("netflix_reviews.csv")  # 파일 불러오기
df = df.iloc[:, 0:5]

# 전처리 함수
def remove_duplicate_emojis(text):
    emoji_pattern = re.compile("[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F]", flags=re.UNICODE)
    emojis = set(emoji_pattern.findall(text))
    for em in emojis:
        text = re.sub(em + '+', em, text)
    return text

def preprocess_text(text):
    if isinstance(text, float):
        return ""
    text = remove_duplicate_emojis(text)
    text = emoji.demojize(text, delimiters=(" ", " "))
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.strip()
    return text

df['content'] = df['content'].apply(preprocess_text)

# 평점 텐서 변환
ratings = torch.tensor(df['score'].values, dtype=torch.long)

# 리뷰 리스트 변환
reviews = df['content'].tolist()

# 토크나이저 및 어휘 사전 생성
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(reviews))

def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

def label_pipeline(label):
    return label

# 데이터셋 클래스 정의
class ReviewDataset(Dataset):
    def __init__(self, reviews, ratings, text_pipeline, label_pipeline):
        self.reviews = reviews
        self.ratings = ratings
        self.text_pipeline = text_pipeline
        self.label_pipeline = label_pipeline

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.text_pipeline(self.reviews[idx])
        rating = self.label_pipeline(self.ratings[idx])
        return torch.tensor(review, dtype=torch.long), torch.tensor(rating, dtype=torch.long)

# 데이터 분할
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(reviews, ratings, test_size=0.2, random_state=42)

# 데이터셋 정의
train_dataset = ReviewDataset(train_reviews, train_ratings, text_pipeline, label_pipeline)
test_dataset = ReviewDataset(test_reviews, test_ratings, text_pipeline, label_pipeline)

# collate_fn 정의
def collate_fn(batch):
    reviews, ratings = zip(*batch)
    reviews = pad_sequence([torch.tensor(r, dtype=torch.long) for r in reviews], batch_first=True, padding_value=0)
    ratings = torch.tensor(ratings, dtype=torch.long)
    return reviews, ratings

# 데이터 로더 정의
BATCH_SIZE = 64
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        embedded = self.embedding(text)
        output, (hidden, cell) = self.lstm(embedded)
        return self.fc(hidden[-1])

# 하이퍼파라미터 정의
VOCAB_SIZE = len(vocab)
EMBED_DIM = 64
HIDDEN_DIM = 128
OUTPUT_DIM = len(set(ratings))

# 모델 초기화
model = LSTMModel(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습 함수 정의
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=20):
    model.train()  # 학습 모드로 설정
    for epoch in range(num_epochs):
        total_loss = 0  # 에포크마다 손실을 추적
        for i, (reviews, ratings) in enumerate(train_dataloader):
            optimizer.zero_grad()
            outputs = model(reviews)  # 모델에 입력하여 예측값 계산
            loss = criterion(outputs, ratings)  # 손실 계산
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            total_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {total_loss/len(train_dataloader):.4f}')
    
    print("Finished Training")



In [8]:
# 모델 학습 실행
train_model(model, train_dataloader, criterion, optimizer, num_epochs=20)



/tmp/ipykernel_170531/906408368.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(review, dtype=torch.long), torch.tensor(rating, dtype=torch.long)
/tmp/ipykernel_170531/906408368.py:85: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reviews = pad_sequence([torch.tensor(r, dtype=torch.long) for r in reviews], batch_first=True, padding_value=0)


KeyboardInterrupt: 

In [ ]:

# 모델 평가

correct = 0
total = 0
with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않음
    for reviews, ratings in test_dataloader:
        reviews, ratings = reviews.to(device), ratings.to(device)
        outputs = model(reviews)
        _, predicted = torch.max(outputs, 1)
        total += ratings.size(0)
        correct += (predicted == ratings).sum().item()

print(f'Accuracy: {100 * correct / total}%')

In [21]:
# 예측 함수
def predict_review(model, review, vocab, tokenizer, device):
    # 리뷰를 텐서로 변환
    tokens = [vocab[token] for token in tokenizer(review)]
    review_tensor = torch.tensor(tokens).unsqueeze(0)  # (1, seq_length) 형태로 만듦
    
    # 텐서를 GPU로 이동
    review_tensor = review_tensor.to(device)
    
    # 모델에 입력하여 예측값 계산
    model.eval()  # 평가 모드로 변경
    with torch.no_grad():  # 평가 시에는 기울기 계산을 하지 않음
        output = model(review_tensor)
        _, predicted = torch.max(output, 1)
    
    return predicted.item()  # 예측된 평점 반환
# 새로운 리뷰에 대한 예측
new_review = "This app is great but has some bugs."
predicted_score = predict_review(model, new_review, vocab, tokenizer,  device)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 5


In [42]:
# 새로운 리뷰에 대한 예측2
new_review = "Good app for streaming occasionally, but this is the only app I have that completely malfunctions my user interface, and forces me to restart my phone. No idea why it happens, but it is profoundly annoying."
predicted_score = predict_review(model, new_review, vocab, tokenizer,  device)
print(f'Predicted Score: {predicted_score}')

Predicted Score: 3


Predicted Score: 4
